#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

2016-04-10 12:33:50,463 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\CHRIST~1\AppData\Local\Temp\graphlab_server_1460309629.log.0


This non-commercial license of GraphLab Create is assigned to ugali_guy@hotmail.com and will expire on March 30, 2017. For commercial licensing options, visit https://dato.com/buy/.


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [4]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [5]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [6]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [7]:
graphlab.canvas.set_target('ipynb')

In [8]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [8]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [14]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [19]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4
products['sentiment'].show('Categorical')

In [20]:
products.head()

name,review,rating,word_count,awesome,great,fantastic
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ...",0,0,0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",0,0,0


##Let's train the sentiment classifier

In [21]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

for word in selected_words:
    products[word] = products['word_count'].apply(lambda word_count_dic: word_count_dic[word] if word in word_count_dic.keys() 
                                                 else 0, dtype = int)
    print 'The number of instances of \"%s\" is %s.' % (word, products[word].sum())
    
products.show()

The number of instances of "awesome" is 2090.
The number of instances of "great" is 45206.
The number of instances of "fantastic" is 932.
The number of instances of "amazing" is 1363.
The number of instances of "love" is 42065.
The number of instances of "horrible" is 734.
The number of instances of "bad" is 3724.
The number of instances of "terrible" is 748.
The number of instances of "awful" is 383.
The number of instances of "wow" is 144.
The number of instances of "hate" is 1220.


In [22]:
train_data,test_data = products.random_split(.8, seed=0)
test_data['sentiment'].show('Categorical')

In [39]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)
print sentiment_model['coefficients'].sort('value', ascending=False).print_rows(num_rows=15, num_columns=10)

print '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
print '======================================================================================================================='
print '-----------------------------------------------------------------------------------------------------------------------'

selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)
print selected_words_model['coefficients'].sort('value', ascending=False).print_rows(num_rows=15, num_columns=10)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 146861

Number of classes           : 2

...

| 4         | 5        | 1.856097     | 0.768747          | 0.767712            |

| 5         | 6        | 2.256122     | 0.768747          | 0.767712            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

+------------+-----------------+-------+---------------+--------+
|    name    |      index      | class |     value     | stderr |
+------------+-----------------+-------+---------------+--------+
| word_count |    label/box.   |   1   | 13.7777956554 |  None  |
| word_count |      hooped     |   1   | 13.1249143709 |  None  |
| word_count | mobile.)update: |   1   |  12.19410881  |  None  |
| word_count |    downgrade.   |   1   |  12.19410881  |  None  |
| word_count |  things/people  |   1   |  11.534921499 |  None  |
| word_count |  direct-pumping |   1   | 11.4331594286 |  None  |
| word_count |      deley      |   1   | 11.0535218292 |  None  |
| word_count |  available.***i |   1   | 10.9520673657 |  None  |
| word_count |     toy!***     |   1   | 10.9520673657 |  None  |
| word_count |      could?     |   1   | 10.7974567626 |  None  |
| word_count |     finallt     |   1   | 10.6930668609 |  None  |
| word_count |     turismo.    |   1   | 10.5445238422 |  None  |
| word_cou

#Evaluate the sentiment model

In [26]:
print "Evaluation of sentiment_model ..."
sentiment_model.evaluate(test_data, metric='roc_curve')

Evaluation of sentiment_model ...


{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |      1.0       | 28005 | 8665 |
 |   1e-05   | 0.923600692441 | 0.996964827709 | 28005 | 8665 |
 |   2e-05   | 0.910675129833 | 0.99642920907  | 28005 | 8665 |
 |   3e-05   | 0.903289094057 | 0.996107837886 | 28005 | 8665 |
 |   4e-05   | 0.899249855741 | 0.995822174612 | 28005 | 8665 |
 |   5e-05   | 0.894402769763 | 0.995679342974 | 28005 | 8665 |
 |   6e-05   | 0.891171379111 | 0.995465095519 | 28005 | 8665 |
 |   7e-05   | 0.887362954414 | 0.995357971791 | 28005 | 8665 |
 |   8e-05   | 0.883554529717 | 0.995322263881 | 28005 | 8665 |
 |   9e-05   | 0.881708020773 | 0.995179432244 | 28005 | 8665 |
 +-----------+----------------+----------------+-------+------

In [27]:
sentiment_model.show(view='Evaluation')

In [28]:
selected_words_model.evaluate(test_data, metric='roc_curve')
selected_words_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Baby Trend Diaper Champ

In [31]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [33]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(baby_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('rating', ascending=False)

In [34]:
print selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

[0.7108950665616598]


In [35]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('rating', ascending=False)
print selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

[0.8866466071266139]


In [36]:
for word in selected_words:
    print diaper_champ_reviews[0][word]

0
0
0
0
1
0
0
0
0
0
0
